<a href="https://colab.research.google.com/github/yinon2592/DL_Project_046211/blob/yinon_branch/section_a_classifier_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
# drive.mount('/content/drive/my-drive/project_calculations')
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:

! pip install transformers
from transformers import set_seed, GPT2Config, GPT2Tokenizer, GPT2ForSequenceClassification
import os
import pandas as pd
from torch.utils.data import Dataset
import re
import torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.3 MB/s eta 0:00:00


In [3]:
model_config = GPT2Config.from_pretrained('gpt2', num_labels=2) # Binary Classification
model = GPT2ForSequenceClassification.from_pretrained('gpt2', config=model_config)

load_model_parameters = True
model_path = '/content/drive/My Drive/project_calculations/model.pth'
if os.path.exists(model_path) and load_model_parameters:
  print("loading last model parameters..")
  model.load_state_dict(torch.load(model_path))

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.padding_side = "left" # Very Important
tokenizer.pad_token = tokenizer.eos_token

model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = model.config.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
test_data_path = '/content/drive/My Drive/project_dataset/test_data.csv'
df = pd.read_csv(test_data_path)
df = df.head(100)
print("dataset size is ", df.shape[0])
print(df.label.value_counts())
print(df.sample(5), "\n")

dataset size is  100
1    55
0    45
Name: label, dtype: int64
    label                                               text
87      0  just wanted to say the youngstown girls miss y...
55      0  i though the clothes of aw0910 collection from...
44      1  no no really its ok u try em out and then u ca...
43      1  stokoe thank you it only took me two minutes t...
9       1  im turning 18 one week from now but i don t fe... 



In [10]:
# Text input for sanity check of sentiment classification
text = "I really enjoyed the movie. It was fantastic!"

# Tokenize the text
encoded_input = tokenizer(text, truncation=True, padding=True, return_tensors='pt')

# Perform sentiment classification
output = model(**encoded_input)

# Retrieve the predicted label and associated probabilities
predicted_label = output.logits.argmax().item()
predicted_probabilities = output.logits.softmax(dim=1).tolist()[0]

# Map the predicted label to sentiment class
sentiment_classes = ["Negative", "Positive"]
predicted_sentiment = sentiment_classes[predicted_label]

# Print the predicted sentiment and associated probabilities
print("Predicted Sentiment:", predicted_sentiment)
print("Sentiment Probabilities:", predicted_probabilities)

Predicted Sentiment: Positive
Sentiment Probabilities: [0.004847042728215456, 0.9951529502868652]


In [11]:
class TweetDataset(Dataset):
    def __init__(self, dataset):
        super().__init__()
        self.data = dataset

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        record = self.data.iloc[index]
        text = record['text']
        return {'text': text, 'label': record['label']}

In [13]:
from torch.utils.data import DataLoader, random_split

test_dataset = TweetDataset(dataset=df)

class Gpt2ClassificationCollator(object):
    def __init__(self, tokenizer, max_seq_len=None):
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len

        return

    def __call__(self, sequences):
        texts = [str(sequence['text']) for sequence in sequences]
        # print(texts)
        labels = [int(sequence['label']) for sequence in sequences]
        # print(labels)
        inputs = self.tokenizer(text=texts,
                                return_tensors='pt',
                                padding=True,
                                truncation=True,
                                max_length=self.max_seq_len)
        inputs.update({'labels': torch.tensor(labels)})

        return inputs

gpt2classificationcollator = Gpt2ClassificationCollator(tokenizer=tokenizer,
                                                        max_seq_len=60)

test_dataloader = DataLoader(dataset=test_dataset,
                            batch_size=32,
                            shuffle=False,
                            collate_fn=gpt2classificationcollator)

In [14]:
def test(dataloader, device_):
    global model
    model.eval()

    prediction_labels = []
    true_labels = []

    total_loss = []

    for batch in dataloader:
        true_labels += batch['labels'].numpy().flatten().tolist()
        batch = {k:v.type(torch.long).to(device_) for k, v in batch.items()}

        with torch.no_grad():
            outputs = model(**batch)
            loss, logits = outputs[:2]
            logits = logits.detach().cpu().numpy()
            total_loss.append(loss.item())

            prediction_labels += logits.argmax(axis=-1).flatten().tolist()

    return true_labels, prediction_labels, total_loss

In [15]:
from sklearn.metrics import classification_report, accuracy_score
import time

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

y, y_pred, val_loss = test(test_dataloader, device)
test_acc = accuracy_score(y, y_pred)

print(test_acc)


0.56
